<h2>Install 시작!!!

In [18]:
!pip install selenium beautifulsoup4 requests webdriver-manager

2561.86s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [19]:
import re
import unicodedata
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import requests
import os

<h3> URL 생성함수

In [20]:
# URL 생성 함수
def get_url(gender='', category=''):
    base_url = 'https://www.musinsa.com/category'
    params = {
        'gf': gender,
        'category': category
    }
    query_string = '&'.join([f"{key}={value}" for key, value in params.items() if value])
    return f"{base_url}/{category}?{query_string}"

# 파일 이름 정리 함수
def clean_filename(filename):
    # Unicode 정규화 (한글 포함 문자 처리 안정성 향상)
    filename = unicodedata.normalize('NFC', filename)
    # 파일 이름에 허용되지 않는 문자 제거
    filename = re.sub(r'[\/:*?"<>|]', '_', filename)
    # 공백과 특수 문자 제거
    filename = re.sub(r'[\n\r\t]', '', filename)
    return filename.strip()

<h3> 이미지 크롤링

In [21]:
# 이미지 크롤링 함수
def crawl_images(driver, gender, category, csv_writer):
    # 성별 및 카테고리 이름 매핑
    gender_map = {'M': '남성', 'F': '여성'}
    category_map = {'002022': '후드집업', '003002': '데님팬츠', '002007': '코트'}

    gender_name = gender_map.get(gender, gender)
    category_name = category_map.get(category, category)

    # 스크롤 설정
    scroll_pause_time = 5
    screen_height = driver.execute_script("return window.screen.height;")
    i = 1

    downloaded_count = 0
    max_downloads = 500  # 최대 다운로드 수 설정

    # 저장 디렉토리 생성
    base_dir = "/home/sejong/Desktop/junseok/SPRC/musinsa/goods"
    output_dir = os.path.join(base_dir, gender_name, category_name)
    os.makedirs(output_dir, exist_ok=True)

    while downloaded_count < max_downloads:
        driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight;")
        if new_height == screen_height * i:
            break
        i += 1

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 상품 목록 추출
        product_elements = soup.select('a[aria-label="상품 상세로 이동"]')
        for product_element in product_elements:
            try:
                # 상품 상세 페이지 URL
                href = product_element.get('href')
                if href.startswith('http'):  # 절대 경로인 경우
                    detail_url = href
                else:  # 상대 경로인 경우
                    detail_url = f"https://www.musinsa.com{href}"

                # 이미지 태그 추출
                img_tag = product_element.select_one('img')
                if not img_tag:
                    continue

                img_url = img_tag.get('src')
                raw_img_name = img_tag.get('alt', 'unknown')
                
                # 파일 이름 정리
                img_name = clean_filename(raw_img_name)
                
                # 이름이 유효하지 않은 경우 기본값 설정
                if not img_name.strip():
                    img_name = f"image_{downloaded_count}"

                if not img_url or not img_name:
                    continue

                if downloaded_count >= max_downloads:
                    print(f"Downloaded {downloaded_count} images. Stopping further downloads.")
                    return
                
                img_path = os.path.join(output_dir, f"{img_name}.jpg")

                # 동일 이름의 파일이 이미 존재하는지 확인
                if os.path.exists(img_path):
                    print(f"이미 동일 이름의 파일이 존재하여 스킵: {img_name}.jpg")
                    continue

                # 이미지 다운로드
                img_response = requests.get(img_url)
                with open(img_path, 'wb') as f:
                    f.write(img_response.content)

                # CSV 파일에 기록
                csv_writer.writerow([img_name, detail_url])
                downloaded_count += 1

                print(f"Downloaded: {img_name}, Detail URL: {detail_url}")
            except Exception as e:
                print(f"Failed to process product: {e}")

    print(f"Downloaded {downloaded_count} images to {output_dir}")

<h3> 크롤링 시작

In [22]:
# 메인 함수
def main():
    genders = ['M', 'F']  # 성별 리스트 (M: 남성, F: 여성)
    categories = ['002022', '003002', '002007']  # 카테고리 리스트 (002022: 후드집업, 003002: 데님팬츠, 002007: 코트)

    # CSV 파일 생성
    csv_file_path = "/home/sejong/Desktop/junseok/SPRC/musinsa/goods/musinsa_data.csv"
    with open(csv_file_path, mode='w', newline='', encoding='utf-8-sig') as csv_file:  # utf-8-sig 적용
        csv_writer = csv.writer(csv_file)
        # CSV 헤더 작성
        csv_writer.writerow(["Image Name", "Product Detail URL"])

        # 웹 드라이버 설정
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

        try:
            for gender in genders:
                for category in categories:
                    if gender == 'M' and category == '002022':
                        continue
                    url = get_url(gender=gender, category=category)
                    print(f"Generated URL: {url}")
                    driver.get(url)
                    crawl_images(driver, gender, category, csv_writer)
        finally:
            driver.quit()

if __name__ == "__main__":
    main()

Generated URL: https://www.musinsa.com/category/003002?gf=M&category=003002
이미 동일 이름의 파일이 존재하여 스킵: M197 wide denim blue.jpg
이미 동일 이름의 파일이 존재하여 스킵: 남성 썸머 라이크라 슬림핏 데님_네이비.jpg
이미 동일 이름의 파일이 존재하여 스킵: Linen Like Relax Fit Denim Pants - Deep Blue.jpg
이미 동일 이름의 파일이 존재하여 스킵: Two Tuck Wide Denim Pants - Black.jpg
이미 동일 이름의 파일이 존재하여 스킵: GP078 크리스 패치드 포켓 데님 팬츠 (BLACK).jpg
이미 동일 이름의 파일이 존재하여 스킵: #0280 mer two tuck wide indigo blue.jpg
이미 동일 이름의 파일이 존재하여 스킵: WORKER PANTS (GREY).jpg
이미 동일 이름의 파일이 존재하여 스킵: 베이직 와이드 데님 팬츠 (BLEACH).jpg
이미 동일 이름의 파일이 존재하여 스킵: Regular Jeans Blue.jpg
이미 동일 이름의 파일이 존재하여 스킵: Linen Like Relax Fit Denim Pants - Black.jpg
이미 동일 이름의 파일이 존재하여 스킵: 커브드 와이드 데님 팬츠 블랙.jpg
이미 동일 이름의 파일이 존재하여 스킵: TAPERED LOOSE FIT DENIM PANTS_L.BLUE.jpg
이미 동일 이름의 파일이 존재하여 스킵: 블랙 커브드 데님 팬츠  블랙.jpg
이미 동일 이름의 파일이 존재하여 스킵: #0345 Gray Wave Wide Flare fit.jpg
이미 동일 이름의 파일이 존재하여 스킵: COVER DENIM PANTS (DENIM).jpg
이미 동일 이름의 파일이 존재하여 스킵: Glossy Pintuck Denim Pants BLACK COATING.jpg
이미 동일 이름의 파일이 존재하여 스킵: 베이직 청바지 

In [23]:
def count_files_in_folder(folder_path):
    """
    폴더 내 파일 개수를 확인하는 함수.

    Parameters:
    - folder_path (str): 파일 개수를 확인할 폴더 경로.
    - include_subfolders (bool): 하위 디렉토리 파일도 포함할지 여부 (기본값: False).

    Returns:
    - int: 폴더 내 파일 개수.
    """
    if not os.path.exists(folder_path):
        print(f"지정된 경로가 존재하지 않습니다: {folder_path}")
        return 0
    
    file_count = 0

    file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

    return file_count

# 사용 예시
for gender in ['남성', '여성']:
    for category in ['데님팬츠', '코트', '후드집업']:

        folder_path = f"/home/sejong/Desktop/junseok/SPRC/musinsa/goods/{gender}/{category}"
        print(f"{gender}/{category} 폴더 내 파일 개수:", count_files_in_folder(folder_path))

남성/데님팬츠 폴더 내 파일 개수: 602
남성/코트 폴더 내 파일 개수: 545
남성/후드집업 폴더 내 파일 개수: 1045
여성/데님팬츠 폴더 내 파일 개수: 545
여성/코트 폴더 내 파일 개수: 545
여성/후드집업 폴더 내 파일 개수: 545


<h3>Done!!